In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader

import torch

import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD

In [2]:
iris = load_iris()

In [3]:
X_train, X_test, y_train, y_test =  train_test_split(iris.data, iris.target, random_state=1)

# Convert to pytorch tensors
X_train, y_train, X_test, y_test = map(
    torch.tensor, (X_train, y_train, X_test, y_test)
)

X_train = X_train.float()
y_train = y_train.long()
X_test = X_test.float()
y_test = y_test.long()


In [4]:
X_train.shape

torch.Size([112, 4])

In [5]:
class IrisDataset(Dataset):
    def __init__(self, X, y):
        #janky but simple
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem(self, idx):
        return {'x': self.X[idx], 'y': self.y[idx] }

In [6]:
class IrisNet(nn.Module):
    def __init__(self, inputshape, outputshape, hiddenunits=10):
        super(IrisNet, self).__init__()
        self.input_layer = nn.Linear(inputshape, hiddenunits)
        self.hidden_layer = nn.Linear(hiddenunits, hiddenunits)
        self.output_layer = nn.Linear(hiddenunits, outputshape)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, X):
        X = F.relu(self.input_layer(X))
        X = self.hidden_layer(X)
        X = self.output_layer(X)
        X = self.softmax(X)
        
        return X

In [7]:
net = IrisNet(4, 3)
net

IrisNet(
  (input_layer): Linear(in_features=4, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=10, bias=True)
  (output_layer): Linear(in_features=10, out_features=3, bias=True)
  (softmax): LogSoftmax()
)

In [20]:
criterion = nn.CrossEntropyLoss()# cross entropy loss
optimizer = SGD(net.parameters(), lr=0.001)


In [22]:
train_ds = TensorDataset(X_train, y_train)


In [23]:
train_ds

In [24]:
y_train

tensor([1, 2, 2, 0, 2, 2, 1, 2, 0, 0, 0, 1, 0, 0, 2, 2, 2, 2, 2, 1, 2, 1, 0, 2,
        2, 0, 0, 2, 0, 2, 2, 1, 1, 2, 2, 0, 1, 1, 2, 1, 2, 1, 0, 0, 0, 2, 0, 1,
        2, 2, 0, 0, 1, 0, 2, 1, 2, 2, 1, 2, 2, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 2,
        2, 2, 0, 0, 1, 0, 2, 0, 2, 2, 0, 2, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0,
        1, 1, 1, 1, 2, 0, 0, 2, 1, 2, 1, 2, 2, 1, 2, 0])

In [25]:
loss_func = F.cross_entropy


In [26]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [27]:
train_dl = DataLoader(train_ds, batch_size=2, shuffle=True)


In [28]:
epochs = 100

In [29]:
for epoch in range(epochs):
    for xb, yb in train_dl:
        pred = net(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(loss_func(net(xb), yb), accuracy(pred, yb))

tensor(0.6805, grad_fn=<NllLossBackward>) tensor(1.)
tensor(1.0091, grad_fn=<NllLossBackward>) tensor(0.5000)
tensor(1.0641, grad_fn=<NllLossBackward>) tensor(0.5000)
tensor(0.9663, grad_fn=<NllLossBackward>) tensor(0.5000)
tensor(0.8552, grad_fn=<NllLossBackward>) tensor(1.)
tensor(0.9222, grad_fn=<NllLossBackward>) tensor(1.)
tensor(0.9212, grad_fn=<NllLossBackward>) tensor(1.)
tensor(1.0020, grad_fn=<NllLossBackward>) tensor(0.5000)
tensor(0.7544, grad_fn=<NllLossBackward>) tensor(1.)
tensor(0.7291, grad_fn=<NllLossBackward>) tensor(1.)
tensor(0.8627, grad_fn=<NllLossBackward>) tensor(0.5000)
tensor(0.9239, grad_fn=<NllLossBackward>) tensor(0.5000)
tensor(0.6266, grad_fn=<NllLossBackward>) tensor(1.)
tensor(0.9200, grad_fn=<NllLossBackward>) tensor(0.5000)
tensor(0.8394, grad_fn=<NllLossBackward>) tensor(0.5000)
tensor(0.8141, grad_fn=<NllLossBackward>) tensor(0.5000)
tensor(0.7730, grad_fn=<NllLossBackward>) tensor(0.5000)
tensor(0.5319, grad_fn=<NllLossBackward>) tensor(1.)
tensor

In [30]:
accuracy(pred, yb)

tensor(1.)

In [31]:
y_test_pred = net(X_test)

In [19]:
y_test

tensor([0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0,
        2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0])

In [39]:
def pred_vs(y_test_pred, y_test):
    preds = torch.argmax(y_test_pred, dim=1)
    return (preds == y_test)

In [40]:
pred_vs(y_test_pred, y_test)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)

In [34]:
y_test_pred

tensor([[ -0.0142,  -4.2595, -11.5616],
        [ -1.7029,  -0.3196,  -2.3924],
        [ -3.6162,  -0.2552,  -1.6178],
        [ -0.0196,  -3.9454, -10.7404],
        [ -7.6268,  -1.0200,  -0.4480],
        [ -4.4146,  -0.4257,  -1.0948],
        [ -6.9802,  -1.0655,  -0.4239],
        [ -0.0352,  -3.3677,  -9.4710],
        [ -0.0338,  -3.4085,  -9.2188],
        [ -8.5853,  -1.6668,  -0.2095],
        [ -3.9737,  -0.4528,  -1.0632],
        [ -0.0212,  -3.8672, -10.5520],
        [ -8.8157,  -1.7118,  -0.1993],
        [ -3.8503,  -0.3083,  -1.4104],
        [ -4.5468,  -0.5127,  -0.9402],
        [ -0.0355,  -3.3598,  -9.0182],
        [ -3.3556,  -0.3151,  -1.4466],
        [ -4.6644,  -0.6159,  -0.7975],
        [ -0.0243,  -3.7292, -10.1792],
        [ -0.0236,  -3.7584, -10.0282],
        [ -4.4469,  -0.5866,  -0.8392],
        [ -4.8141,  -0.6988,  -0.7038],
        [ -5.5766,  -0.6504,  -0.7457],
        [ -0.0248,  -3.7120, -10.0200],
        [ -7.7542,  -1.2624,  -0.3333],


In [35]:
 torch.argmax(y_test_pred, dim=1)

tensor([0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0,
        2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0])

In [43]:
preds = torch.argmax(y_test_pred, dim=1)
preds

tensor([0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0,
        2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0])

In [42]:
y_test

tensor([0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0,
        2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0])